# 1. 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [2]:
import pickle

In [190]:
service = Service('chromedriver.exe')
driver = webdriver.Chrome(service = service)

In [4]:
isbn13 = pd.read_pickle('isbn13.pkl')

In [6]:
isbn13

0        9791190710343
1        9791190710350
2        9791190710367
3        9791128858338
4        9788954685603
             ...      
54971    9791185020136
54972    9788997838127
54973    9788956591858
54974    9788987794938
54975    9788995696972
Name: isbn13, Length: 54976, dtype: object

In [ ]:
rank_mean = [] # 도서 평균 평점
rank_percentage = [] # 평균 percentage
review_title = [] # 리뷰 제목
review_id = [] # 리뷰 id
review_rank = [] # 리뷰 평점
review_text = [] # 리뷰 
isbn_idx = []


for isbn in isbn13[5500:6000]: # 페이지 입력
    url = "http://www.yes24.com/product/search?query={}".format(isbn)
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 페이지에 있는 도서 링크(50개)
    book_links = soup.select('a.gd_name')
    goods = []
    for link in book_links:
        a= link['href'].split('/')[3] 
        goods.append(a)
    for item in goods:
        del(goods[1:])
        driver.get('http://www.yes24.com'+'/Product/Goods/{}'.format(item)) # 상품 페이지
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        try:
            mean = soup.select('div.gd_rating > em.yes_b')[0].text
            rank_mean.append([mean]) # 도서 평균 평점
        except:
            mean = []
            rank_mean.append(mean) # 도서 평균 평점
        rank_p = []
        try:
            for k in range(5):
                rank_ = soup.select('span.barGraphBase')[k].text # 평점 percentage
                rank_p.append(rank_)
            rank_percentage.append(rank_p)
        except:
            rank_ = []
            rank_percentage.append(rank_)
        try:
            review_t = []  # 각각의 책의 리뷰제목 리스트
            review_i = []
            review_r = []
            review_te = []
            for page in range(1,3): # 리뷰 페이지 2개
                driver.get('http://www.yes24.com/Product/communityModules/GoodsReviewList/{}?Type=ALL&_=1648001707174&Sort=2&PageNumber={}'.format(item, page))
                time.sleep(1)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                
                blog_link = []
                for i in range(5): # 리뷰 한 페이지에 있는 5개 리뷰 가져오기
                    try:
                        title = soup.select('span.review_tit')[i].text.strip() # 리뷰 제목
                        review_t.append(title)
                        r_id = soup.select('a.lnk_id')[i].text # 리뷰 id
                        review_i.append(r_id)
                        rank = soup.select('span.review_rating')[i].text.split('\n')[1][-2:] # 리뷰 평점
                        review_r.append(rank)
                        blog_l = soup.select('div.review_lnk a')[i]['href'] # 리뷰 블로그 링크
                        blog_link.append(blog_l)
                    except:
                        pass
                for j in range(5): # 리뷰 한 페이지에 있는 5개 블로그 내용 가져오기
                        driver.get(blog_link[j]) # 리뷰 블로그 링크 들어가기
                        time.sleep(1)
                        html = driver.page_source
                        soup = BeautifulSoup(html, 'html.parser')
                        text = soup.select('div.blogContArea')[0].text # 리뷰 내용 
                        review_te.append(text)
        except:
            pass
        review_title.append(review_t)
        review_id.append(review_i)
        review_rank.append(review_r)
        review_text.append(review_te)
        isbn_idx.append(isbn)

In [189]:
col = ['isbn','rank_mean','rank_percentage','review_title','review_id','review_rank','review_text']
user = pd.DataFrame([isbn_idx,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text]).T
user.columns = col
user

,isbn,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text
0,9788932918952,[8.0],[],[],[],[],[]
1,9788960902756,[],[],[],[],[],[]
2,9791187878001,[],[],[],[],[],[]
3,9788932818801,[10.0],"[100%, 0%, 0%, 0%, 0%]","[모든 [엄마의 엄마]에게 / 이주현, 어른을 위한 동화책 '엄마의 엄마', 엄마...","[t*******4, j*******u, a*****4, 아*르, 아***쉬, b*...","[5점, 5점, 5점, 5점, 5점, 5점]",[\n#도서협찬\n \n이주현 글. 그림\n\n안녕하세요 동그리 독서입니다.^^\n...
4,9791164455126,[],[],[],[],[],[]
...,...,...,...,...,...,...,...
79,9788958870241,[8.9],"[66%, 30%, 4%, 0%, 0%]","[술냄새가 아닌 사람냄새가 진동하는 술통, ''술과 함께 한 철학자 장승욱'' <젊...","[쩡*, 연**경, 햇**이, 지***0, j******e, e*****k, 꽃*지...","[5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 4점, 5점]","[\n술. 고등학생 때까지만 해도 나에게 술은 마약, 본드, 담배처럼 절대 가까이 ..."
80,9791185346731,[9.6],"[89%, 11%, 0%, 0%, 0%]","[달빛 변호사, 마음을 여는 변론 달빛 변호사, 달빛 변호사, 마음을 여는 변론, ...","[j*****n, 세**럼, 흔**자, 매*얼, 변*사, 한*, 캡***크, 초**...","[5점, 5점, 5점, 5점, 5점, 5점, 5점, 4점, 4점]",[\n 이 책을 읽으면서 머릿속에서는 드라마 굿 와이프의 법정 장면들이 자연히 떠오...
81,9791189877613,[9.8],"[94%, 0%, 6%, 0%, 0%]","[씩씩한 항암녀의 속엣말, 씩씩한 항암녀의 속엣말, 씩씩한 항암녀의 속·엣·말, [...","[엉**어, c****k, j***e, b*****s, 밀*티, 청*, m*****...","[5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점]",[\n라디오 DJ 등으로 활동 중이던 저자는 유방암 판정을 받고 치료하면서 그녀의 ...
82,9788925563466,[8.0],"[0%, 100%, 0%, 0%, 0%]",[- 조금은 달라도 충분히 행복하게],[호**이],[4점],[\n 도시에서 10년간 패션 매거진 에디터로 바쁘고 화려하게 살아왔은 저자가 ...


In [183]:
for i in range(2955,3041):
    if isbn13[i] == '9791191994025':
        print(i)

3040


In [184]:
user.to_pickle('user_2955-3040.pkl', protocol = 4)

In [120]:
user1 = pd.read_pickle('user_1700-1986..pkl') 

In [187]:
user1 = pd.read_pickle('user_4474-4680.pkl')
user2 = pd.read_pickle('user_4681-4731.pkl')


user3 = pd.read_pickle('user_6000-6010.pkl')
user4 = pd.read_pickle('user_6011-6043.pkl')
user4 = pd.read_pickle('user_6044-6088.pkl')
#user4 = pd.read_pickle('user_6011-6043.pkl')
#user4 = pd.read_pickle('user_6011-6043.pkl')
#user4 = pd.read_pickle('user_6011-6043.pkl')

In [188]:
df = pd.concat([user1, user2],ignore_index=True)
df.to_pickle('user_4474-4731.pkl', protocol = 4)

In [ ]:
4473~6999